## **Inspect Model**

### **Imports**

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys

ROOT_DIR = os.path.abspath("../")
print('Root directory:',ROOT_DIR)

sys.path.append(ROOT_DIR)
import mrcnn.model as modellib

%matplotlib inline 

LOGS_DIR = os.path.join(ROOT_DIR, "logs", "inspect")

if not os.path.exists(LOGS_DIR):
    os.makedirs(LOGS_DIR)
    print(f"Folder '{LOGS_DIR}' created.")

Root directory: c:\Users\abell\Documents\aa_delft\J4\BEP\ai


In [2]:
%load_ext autoreload
%autoreload 2
import bep.utils as utils
from bep.dataset import bepDataset

### **Configurations**

In [3]:
import tdmms.tdmcoco as tdmcoco
config = tdmcoco.CocoConfig()

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 3  # thin, few, thick
    DETECTION_MIN_CONFIDENCE = 0.7
    USE_MINI_MASK = False
config = InferenceConfig()
# config.display()

### **Load data**

In [4]:
utils.create_dir_setup(ROOT_DIR, (0.8, 0.1, 0.1))

Creating directories from batches
Found batches: batch1, batch2, batch3, batch4
Images forced for training directory: [('batch4', '67_sio2_NbSe2_Exfoliation_B2-31_f1_img.png'), ('batch4', '67_sio2_NbSe2_Exfoliation_B2-42_f2_img.png'), ('batch4', '67_sio2_NbSe2_Exfoliation_B2-52_f3_img.png'), ('batch4', '67_sio2_NbSe2_Exfoliation_C5-84_f4_img.png')]
Total image count: 36
Copying images
Checking image counts
Creating and writing annotation files


In [5]:
utils.check_dir_setup(ROOT_DIR, (0.8, 0.1, 0.1))

train, _, test = utils.load_train_val_datasets(ROOT_DIR)

split = bepDataset()
split.load_dir(os.path.join(ROOT_DIR, 'data'), 'batchsplit', reload_annotations=False)
split.prepare()

print("BEP Train NbSe2 images: {} | Classes: {}".format(len(train.image_ids), train.class_names))
print("BEP Test NbSe2 images: {} | Classes: {}".format(len(test.image_ids), test.class_names))
print("BEP Split NbSe2 images: {} | Classes: {}".format(len(split.image_ids), split.class_names))

Directory setup correctly
BEP Train NbSe2 images: 85 | Classes: ['BG', 'Mono_NbSe2', 'Few_NbSe2', 'Thick_NbSe2']
BEP Test NbSe2 images: 3 | Classes: ['BG', 'Mono_NbSe2', 'Few_NbSe2', 'Thick_NbSe2']
BEP Split NbSe2 images: 60 | Classes: ['BG', 'Mono_NbSe2', 'Few_NbSe2', 'Thick_NbSe2']


### **Load Model**

In [6]:
BEP_MODEL_PATH = os.path.join(ROOT_DIR, 'weights', '20241118-133739_nbse2_mos2_4_True_89_2__0096.h5')
# BEP_MODEL_PATH = os.path.join(ROOT_DIR, 'weights', '20241105-095909_nbse2_mos2_1_True_33_2__0014.h5')


model_NbSe2 = modellib.MaskRCNN(mode="inference", model_dir=LOGS_DIR, config=config, name='NbSe2 Fine-tuned Model')
print("NbSe2 Model, loading weights {}".format(BEP_MODEL_PATH))
model_NbSe2.load_weights(BEP_MODEL_PATH, by_name=True)

Instructions for updating:
Use fn_output_signature instead
NbSe2 Model, loading weights c:\Users\abell\Documents\aa_delft\J4\BEP\ai\weights\20241118-133739_nbse2_mos2_4_True_89_2__0096.h5


### **Run Model**

In [7]:
run_model = utils.runModel(model_NbSe2, config, plot_size=15)

image ID: ali.1 (0) 
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  245.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  121.30000  float64
image_metas              shape: (1, 16)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


In [23]:
run_model.run(dataset=test, image_idx=0)

In [24]:
run_model.gt()